# Create an animated sequence of images from an image collection

In this notebook we will compute the average normalize difference vegetation index (NDVI) for each day of the year using data for the past 10 years.

**Task**: Run the code cells below and try to follow the reasoning. The code creates an animation for Kansas. Your job is to create an animation for a different US state or a different country.

If you decide on a country, consider the FAO dataset for administrative boundaries: https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_SIMPLIFIED_500m_2015_level1?hl=en#table-schema

You need to upload the modified code and the resulting .GIF file.

**Expected result**: A GIF image showing the sequence of images for the year.

**Due date**: Assignment is due on Thursday, August 4rd at 8 AM. Please, upload to canvas the modified Jupyter notebook and the resulting animation.


In [1]:
# Import and initialize GEE
import ee
ee.Initialize()


In [2]:
# Define a region (Need to modify this if selecting a state other than Kansas)
# Four numbers in the order xMin, yMin, xMax, yMax.
rect = ee.Geometry.Rectangle([[-103,36.5], [-94,40.5]]);


In [3]:
# Select a collection from the available dataset
collection = ee.ImageCollection('MODIS/006/MOD13A2').select('NDVI')


In [4]:
# Select a range of dates and a specific band
vegetation = collection.filterDate('2012-01-01', '2022-01-01');


In [5]:
# Select state boundary
# For countries you can use: FAO/GAUL_SIMPLIFIED_500m/2015/level1
# A site with country codes: http://www.statoids.com/wab.html

region = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME', 'Kansas'))



In [6]:
# Obtained images from the collection for each day of the year.
# Note that we need to define a function for each image and then pass this function to GEE.
# GEE automatically iterates over all the images in the collection (this is what the 'map()' method does.

# Define function
def get_doy(img):
    doy = ee.Date(img.get('system:time_start')).getRelative('day', 'year')
    return img.set('doy', doy)

# Call map and apply the function to each image
collection = collection.map(get_doy)


In [7]:
# Filter the complete collection to a single year of data e.g. 2021.
# We use one year as a dummy variable to compute the day of the year.
distinctDOY = collection.filterDate('2021-01-01', '2022-01-01')


In [8]:
# Define a filter that identifies which images from the complete collection
# match the DOY from the distinct DOY collection.
# leftField == rightField
filt = ee.Filter.equals(leftField='doy', rightField='doy')


In [9]:
# Define a join.
join = ee.Join.saveAll('doy_matches')

# Apply the join and convert the resulting FeatureCollection to an ImageCollection.
joinCol = ee.ImageCollection(join.apply(distinctDOY, collection, filt))


In [10]:
# Apply median reduction among matching DOY collections.
def apply_median(img):
    doyCol = ee.ImageCollection.fromImages(img.get('doy_matches'))
    return doyCol.reduce(ee.Reducer.median()).multiply(0.0001)

composite = joinCol.map(apply_median)


In [11]:
# Create animation

# Function that handles the visuals (paint adds the boundary, 
# which is a assigned a value (-0.1) relative to the other pixels (so a low value means red here).
def animate(img):
    cmap = ['red','FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
        '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
        '012E01', '011D01', '011301']
    frame = img.paint(region,-0.1,2).visualize(min=-0.1, max=0.8, palette=cmap)
    return frame

# Map the function to each image
animation = composite.map(animate)


In [12]:
# Animation options
animationOptions = {'region': rect,     # Selected region on the map
                    'dimensions': 600,    # Size of the animation
                    'crs': 'EPSG:3857',   # Coordinate reference system
                    'framesPerSecond': 6  # Animation speed
}

In [13]:
# Render the GIF animation in the console.
print(animation.getVideoThumbURL(animationOptions))


https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/06365075ae1da38db7e2d880bb4f7221-b3fb45bed9f1a3f692b0ff5194918715:getPixels


In [ ]:
# Right click on the generated GIF image in the browser and select "save image as" to download it.
